# Data Preparation: Cleaning of PSA Food Price Data

This notebook documents the unified process for loading, cleaning, and transforming the raw food price data sourced from the Philippine Statistics Authority (PSA). 
The goal is to convert the wide, aggregated raw files into a single, clean, and tidy (long) provincial-level dataset ready for time-series analysis.

In [5]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

RAW_DATA_DIR = "raw_dataset"
CLEANED_DATA_DIR = "data_cleaned"

RAW_PRICE_FILES = [
    "Condiments-Food-Prices.csv",
    "Fruit-Vegetables-Food-Prices.csv",
    "Fruits-Food-Prices.csv",
    "Leafy-Vegetables-Food-Prices.csv",
    "Rootcrops-Food-Prices.csv"
]

os.makedirs(CLEANED_DATA_DIR, exist_ok=True)

In [7]:
def clean_and_save_data(file_name: str, raw_folder: str = RAW_DATA_DIR, output_folder: str = CLEANED_DATA_DIR):

    full_path = os.path.join(raw_folder, file_name)

    try:
        df = pd.read_csv(full_path, skiprows=2, header=0)
    except FileNotFoundError:
        print(f"File not found: {full_path}")
        return

    df[['Commodity_Name', 'Unit']] = df['Commodity'].str.rsplit(', ', n=1, expand=True)
    df = df.drop(columns=['Commodity'])

    id_vars = ['Geolocation', 'Commodity_Name', 'Unit']
    price_cols = df.columns.difference(id_vars).tolist()

    df[price_cols] = df[price_cols].replace(['..', '.'], np.nan)
    df[price_cols] = df[price_cols].apply(pd.to_numeric, errors='coerce')

    all_nan_cols = df.columns[df.isna().all()].tolist()
    if all_nan_cols:
        df = df.drop(columns=all_nan_cols)

    value_vars = df.columns.difference(id_vars).tolist()

    def time_sort_key(time_str):
        try:
            return datetime.strptime(time_str, '%Y %B')
        except ValueError:
            return datetime(1900, 1, 1)

    value_vars.sort(key=time_sort_key)

    df_long = pd.melt(df, id_vars=id_vars, value_vars=value_vars,
                      var_name='Time_Period', value_name='Retail_Price')
    df_long['Geolocation'] = df_long['Geolocation'].str.strip()

    df_long['dots'] = df_long['Geolocation'].str.extract(r'^(?P<dots>\.+)')['dots'].str.len()
    df_long['dots'] = df_long['dots'].fillna(0).astype(int)

    df_long['Region'] = np.where(df_long['dots'] == 2,
                                 df_long['Geolocation'].str.lstrip('.').str.strip(), np.nan)
    df_long['Province'] = np.where(df_long['dots'] == 4,
                                   df_long['Geolocation'].str.lstrip('.').str.strip(), np.nan)

    df_long['Region'] = df_long['Region'].ffill()

    df_long = df_long.dropna(subset=['Province']).reset_index(drop=True)

    df_long = df_long.drop(columns=['dots', 'Geolocation'])

    df_long[['Year', 'Month']] = df_long['Time_Period'].str.split(' ', expand=True)
    df_long['Year'] = df_long['Year'].astype(int)

    df_long = df_long[['Region', 'Province', 'Commodity_Name', 'Unit', 'Year', 'Month', 'Retail_Price']]

    output_file = os.path.join(output_folder, f"{file_name.replace('.csv','')}.csv")
    df_long.to_csv(output_file, index=False)
    print(f" Cleaned data saved: {output_file}. Final shape: {df_long.shape}")

In [9]:
for f_name in RAW_PRICE_FILES:
    clean_and_save_data(f_name)

/var/folders/kk/r__l3n2d23s7_2w_328wzkpc0000gn/T/ipykernel_3446/3868427973.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[price_cols] = df[price_cols].replace(['..', '.'], np.nan)
/var/folders/kk/r__l3n2d23s7_2w_328wzkpc0000gn/T/ipykernel_3446/3868427973.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[price_cols] = df[price_cols].replace(['..', '.'], np.nan)


 Cleaned data saved: data_cleaned/Condiments-Food-Prices.csv. Final shape: (22400, 7)
 Cleaned data saved: data_cleaned/Fruit-Vegetables-Food-Prices.csv. Final shape: (50400, 7)


/var/folders/kk/r__l3n2d23s7_2w_328wzkpc0000gn/T/ipykernel_3446/3868427973.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[price_cols] = df[price_cols].replace(['..', '.'], np.nan)


 Cleaned data saved: data_cleaned/Fruits-Food-Prices.csv. Final shape: (78400, 7)
 Cleaned data saved: data_cleaned/Leafy-Vegetables-Food-Prices.csv. Final shape: (28000, 7)


/var/folders/kk/r__l3n2d23s7_2w_328wzkpc0000gn/T/ipykernel_3446/3868427973.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[price_cols] = df[price_cols].replace(['..', '.'], np.nan)
/var/folders/kk/r__l3n2d23s7_2w_328wzkpc0000gn/T/ipykernel_3446/3868427973.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[price_cols] = df[price_cols].replace(['..', '.'], np.nan)


 Cleaned data saved: data_cleaned/Rootcrops-Food-Prices.csv. Final shape: (33600, 7)
